In [1]:
import numpy as np
import pandas as pd


In [2]:
train = pd.read_csv(r'C:\Users\rlee3104\Desktop\HDFC\d407ebe0db9a11e9\DataSet\Train.csv')

C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (746,835) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
trainX = pd.read_csv(r'C:\Users\rlee3104\Desktop\HDFC\d407ebe0db9a11e9\DataSet\Train.csv')
trainX['Col2'].value_counts()

0    15760
1     1761
Name: Col2, dtype: int64

In [4]:
test = pd.read_csv(r'C:\Users\rlee3104\Desktop\HDFC\d407ebe0db9a11e9\DataSet\Test.csv')

C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (700,731,740,752,761,789,811,820,829,841,850) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt 
from scipy.stats import norm, skew 

In [6]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.Col2.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['Col2'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


all_data size is : (37963, 2394)


In [7]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(1000)

,Missing Ratio
Col266,99.957854
Col288,99.957854
Col813,99.957854
Col485,99.957854
Col724,99.957854
Col463,99.957854
Col171,99.141269
Col930,97.884783
Col40,97.745173
Col566,96.957564


In [8]:
all_data = all_data.loc[:, all_data.isnull().mean() < .70]

In [9]:
all_data = all_data.fillna(all_data.mean())

In [10]:
all_data.drop(['Col1'], axis=1,inplace= True)


In [11]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(500)


Skew in numerical features: 



,Skew
Col515,191.330787
Col513,190.838192
Col435,190.759172
Col665,190.678296
Col663,189.993690
Col631,189.922024
Col179,180.788394
Col22,170.226486
Col332,163.298663
Col529,163.209865


In [12]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)
    

There are 2200 skewed numerical features to Box Cox transform


In [13]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [14]:
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(
    train, y_train, test_size=0.1, random_state=42
)

In [15]:
X_train[:] = np.nan_to_num(X_train)

C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3374: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer(key, value)
C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: h

In [16]:
X_test[:] = np.nan_to_num(X_test)

C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [22]:
test = sc_X.transform(test)

In [24]:
test[:] = np.nan_to_num(test)

In [60]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from vecstack import stacking
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import lightgbm as lgbClassifier

In [ ]:
application: 'binary',objective: 'binary',metric: 'auc',is_unbalance: 'true',boosting: 'gbdt',num_leaves: 31,feature_fraction: 0.5,learning_rate: 0.05
    application: 'binary',objective: 'binary',metric: 'auc',is_unbalance: 'true',boosting: 'gbdt',num_leaves: 31,
                            

In [68]:
models = [

  #  lgbClassifier() ,
    ExtraTreesClassifier(random_state=0, n_jobs=-1, 
                         n_estimators=500, max_depth=8),
        
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=500, max_depth=3),
    
    
    #KNeighborsClassifier(n_neighbors=5),
    LogisticRegression(),
    SVC(kernel="linear", C=0.025)
   # SVC(gamma=2, C=1)
    
    
    
]

In [69]:
S_train, S_test = stacking(models,                     # list of models
                           X_train, y_train, test,   # data
                           regression=False,           # classification task (if you need 
                                                       #     regression - set to True)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and vote
                           needs_proba=False,          # predict class labels (if you need 
                                                       #     probabilities - set to True) 
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=accuracy_score,      # metric: callable
                           n_folds=4,                  # number of folds
                           stratified=True,            # stratified split for folds
                           shuffle=True,               # shuffle the data
                           random_state=0,             # ensure reproducibility
                           verbose=2)  

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [4]

model  0:     [ExtraTreesClassifier]
    fold  0:  [0.72685772]
    fold  1:  [0.72001014]
    fold  2:  [0.73255519]
    fold  3:  [0.74194367]
    ----
    MEAN:     [0.73034168] + [0.00803717]
    FULL:     [0.73033993]

model  1:     [RandomForestClassifier]
    fold  0:  [0.89906163]
    fold  1:  [0.89906163]
    fold  2:  [0.89926415]
    fold  3:  [0.89926415]
    ----
    MEAN:     [0.89916289] + [0.00010126]
    FULL:     [0.89916286]

model  2:     [LogisticRegression]


C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


    fold  0:  [0.71214811]


C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


    fold  1:  [0.66573675]


C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


    fold  2:  [0.67876174]


C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


    fold  3:  [0.71200203]
    ----
    MEAN:     [0.69216216] + [0.02043852]
    FULL:     [0.69216134]

model  3:     [SVC]
    fold  0:  [0.68247527]
    fold  1:  [0.65584580]
    fold  2:  [0.66150723]
    fold  3:  [0.67419437]
    ----
    MEAN:     [0.66850567] + [0.01044942]
    FULL:     [0.66850583]



In [70]:
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.01, 
                      n_estimators=1000, max_depth=8,scale_pos_weight = 7)
    
# Fit 2nd level model
model = model.fit(S_train, y_train)

# Predict
y_pred = model.predict(S_test)



In [71]:
y_pred1 = model.predict(S_train)

In [72]:
np.unique(y_pred)

array([0, 1], dtype=int64)

In [73]:
testing = pd.read_csv(r'C:\Users\rlee3104\Desktop\HDFC\d407ebe0db9a11e9\DataSet\Test.csv')

C:\Users\rlee3104\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (700,731,740,752,761,789,811,820,829,841,850) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [76]:
my_submission = pd.DataFrame({'Col1': testing.Col1, 'Col2': y_pred})
# you could use any filename. We choose submission here
my_submission.to_csv('HDFC_backup.csv', index=False)

In [74]:
from termcolor import colored
#from sklearn.cross_validation import KFold

from sklearn.metrics import confusion_matrix

In [75]:
#y_pred1 = model.predict(dte)
# Making the Confusion Matrix
cm = confusion_matrix(y_train, (y_pred1>0.5))
print(colored('The Confusion Matrix is: ', 'red'),'\n', cm)
# Calculate the accuracy on test set
predict_accuracy_on_test_set = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1]+cm[1,0] + cm[0,1])
print(colored('The Accuracy on Test Set is: ', 'blue'), colored(predict_accuracy_on_test_set, 'blue'))

The Confusion Matrix is:  
 [[10127  4051]
 [  702   888]]
The Accuracy on Test Set is:  0.6985667174023339
